In [12]:
import torch
from Transformer import GPT, GPTConfig
from tqdm import tqdm
from  data_provider import dataloader_train, dataloader_valid
import pandas as pd
import numpy as np


In [13]:
from torch.optim.lr_scheduler import StepLR

# Define model, optimizer, and scheduler
model = GPT(GPTConfig)
learning_rate = 2e-8
num_epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)  # Decays LR by 0.1 every 3 epochs

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

model.train()
loss_fn = torch.nn.MSELoss()

# Training loop
for epoch in tqdm(range(num_epochs)):
    train_losses = []
    for batch_idx, (X, y) in enumerate(dataloader_train):  # [window_size, n_features]
        X = X.to(device, dtype=torch.float)
        y = y.to(device, dtype=torch.double)
        out = model(X.to(torch.float)).squeeze()
        y = y.to(torch.double)
        out = out.to(torch.double)
        loss = loss_fn(out, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_losses.append(loss.data.cpu())

    # Update learning rate using the scheduler
    scheduler.step()

    print('Epoch %d: Current mean loss = %.4f, Learning Rate = %.2e' % 
          (epoch, np.mean(train_losses, axis=0), scheduler.get_last_lr()[0]))
    
torch.save(model.state_dict(), "model_state.pth")
print('Finished Training and Model Saved to model_state.pth')


 10%|█         | 1/10 [1:34:40<14:12:06, 5680.76s/it]

Epoch 0: Current mean loss = 1.0159, Learning Rate = 2.00e-08


 20%|██        | 2/10 [1:57:00<6:56:58, 3127.28s/it] 

Epoch 1: Current mean loss = 0.9814, Learning Rate = 2.00e-08


 20%|██        | 2/10 [2:28:12<9:52:48, 4446.09s/it]


KeyboardInterrupt: 